In [1]:
from __future__ import print_function

import os
from datetime import datetime, date, time, timezone
import numpy as np
import matplotlib.pyplot as plt
#import ipywidgets as widgets
#import IPython.display

import pyTMD.io
import pyTMD.time
import pyTMD.predict
import pyTMD.tools
import pyTMD.utilities

# autoreload
%load_ext autoreload
%autoreload 2

C:\Users\sibekwut\Anaconda3\lib\site-packages\pyTMD\spatial.py:70: UserWarning: GDAL not available
  warnings.warn("GDAL not available")
C:\Users\sibekwut\Anaconda3\lib\site-packages\pyTMD\spatial.py:71: UserWarning: Some functions will throw an exception if called
  warnings.warn("Some functions will throw an exception if called")
<frozen importlib._bootstrap>:228: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 16 from C header, got 88 from PyObject


In [9]:
# available model list
FORMAT = 'netcdf'

if FORMAT == 'netcdf':
    model = pyTMD.io.model('C:\\Users\\sibekwut\\OneDrive - University of New Brunswick\\Fall 2022', format='netcdf',
                compressed=False).elevation('TPXO9-atlas-v5')
elif FORMAT == 'FES2014':
    model = pyTMD.io.model('C:\\Users\\sibekwut\\OneDrive - University of New Brunswick\\Fall 2022', format='FES',
                compressed=False).elevation('FES2014')


D = date(2022, 6, 2)
T = time(1, 54, 36,990000)
t = datetime.combine(D,T)

LAT,LON = (50.13769298831343,-59.272468344843766)

dates = np.datetime64(t)
print(dates)
print(t)

2022-06-02T01:54:36.990000
2022-06-02 01:54:36.990000


In [5]:
#calculate one minute forecast for every second

# For any of the method you're usiing make sure you're using the right epoch

#The Calendaar date module of converting time does not accept thee microseconds part of the time

#tide_epoch = 1992-01-01T00:00:00

#tide_time1 = pyTMD.time.convert_calendar_dates(t.year, t.month, t.day, t.hour, t.minute, t.second, \
                                               #epoch=pyTMD.time._tide_epoch, scale=1.0)

#The date time conversion accepts the microseconds aspect of the date
tide_time = pyTMD.time.convert_datetime(dates, epoch =pyTMD.time._tide_epoch)/86400


#tide_times1 = np.array([tide_time1])
tide_times = np.array([tide_time])

# number of time points
nt = len(tide_times)

# delta time (TT - UT1) file
delta_file = pyTMD.utilities.get_data_path(['data','merged_deltat.data'])


#print('Tide_time using calendar_dates: ', tide_times1)
print('Tide_time using datetime: ', tide_times)
print(delta_file)
print(nt)

Tide_time using datetime:  [11110.07959479]
C:\Users\sibekwut\Anaconda3\lib\site-packages\pyTMD\data\merged_deltat.data
1


In [10]:
if model.format == 'netcdf':
    constituents = pyTMD.io.ATLAS.read_constants(
        model.grid_file, model.model_file,
        type=model.type, compressed=model.compressed)
    c = constituents.fields
    DELTAT = np.zeros_like((nt))

elif model.format == 'FES':
    constituents = pyTMD.io.FES.read_constants(model.model_file,
        type=model.type, version=model.version,
        compressed=model.compressed)
    c = model.constituents
    # interpolate delta times from calendar dates to tide time
    DELTAT = pyTMD.time.interpolate_delta_time(delta_file, tide_times)
    
    print(DELTAT)

In [11]:
# update the tide prediction and plot
def update_tide_prediction():
    if model.format == 'netcdf':
        amp,ph,D = pyTMD.io.ATLAS.interpolate_constants(
            np.atleast_1d(LON), np.atleast_1d(LAT),
            constituents, type=model.type, scale=model.scale,
            method='spline', extrapolate=True)
    
    elif model.format in 'FES':
        amp,ph = pyTMD.io.FES.interpolate_constants(
            np.atleast_1d(LON), np.atleast_1d(LAT),
            constituents, scale=model.scale,
            method='spline', extrapolate=True)
        
    # calculate complex phase in radians for Euler's
    cph = -1j*ph*np.pi/180.0
    # calculate constituent oscillation
    hc = amp*np.exp(cph)
    # predict tidal elevations at time 1 and infer minor corrections
    TIDE = pyTMD.predict.drift(tide_times, hc, c,
        deltat=DELTAT, corrections=model.format)
    MINOR = pyTMD.predict.infer_minor(tide_times, hc, c,
        deltat=DELTAT, corrections=model.format)
    TIDE.data[:] += MINOR.data[:]
    # convert to meters
    TIDE.data[:] *= 1
    
    print("Time= ", dates)
    print("Longitude= ", LON)
    print("Latitude= ", LAT)
    print("Tide=", TIDE)
    
# run tide prediction at initial location
update_tide_prediction()

Time=  2022-06-02T01:54:36.990000
Longitude=  -59.272468344843766
Latitude=  50.13769298831343
Tide= [0.3084135378521191]
